In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential

In [2]:
model_2=keras.models.Sequential([
    #keras.layers.Flatten(input_shape=(227,227,3)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(227,227,3)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), activation='relu', padding="same"),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(27,activation='softmax') 
])
    
    
model_2.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 225, 225, 128)     147584    
                                                                 
 batch_normalization (Batch  (None, 225, 225, 128)     512       
 Normalization)                                                  
                                                                 
 conv2d_2 (Conv2D)           (None, 225, 225, 256)     295168    
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 256)     0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 256)    

In [3]:
model_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
#Re-Load the dataset
loaded_data = np.load("my_dataset_aug_full.npz", allow_pickle=True)

# Access images and labels
loaded_images = loaded_data['images']

# Now you can use loaded_images and loaded_labels in your code
image_df=pd.DataFrame(loaded_images)

#split it into features and labels
features = image_df.iloc[:, :51529]
labels = image_df.iloc[:, 51530]
labels_coded=image_df.iloc[:,51531]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels_coded, test_size=0.2, random_state=42)

# Display the shapes of the resulting splits
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)



X_train shape: (7925, 51529)
X_test shape: (1982, 51529)
y_train shape: (7925,)
y_test shape: (1982,)


In [6]:
#try-outs with smaller subsets:
X_train_subset = X_train[:100]
y_train_subset = y_train[:100]
X_test_subset = X_test[:100]
y_test_subset = y_test[:100]

In [7]:
X_train_subset=X_train_subset/255.0
X_test_subset=X_test_subset/255.0

In [ ]:
one_pic = X_train_subset.iloc[0]
one_pic_array = one_pic.values.astype(float)  # Convert to float
one_pic_image = one_pic_array.reshape((227, 227))
one_pic_image.shape

In [8]:
def image_reshape(image_df):
    images = []
    for idx, row in image_df.iterrows():
        one_pic_array = row.values.astype(float)
        one_pic_image = one_pic_array.reshape((227, 227))
        images.append(one_pic_image)
    return np.array(images)

reshaped_images = image_reshape(X_train_subset)
reshaped_images.shape

(100, 227, 227)

In [ ]:
import torch

reshaped_images_tensor = torch.tensor(reshaped_images, dtype=torch.float32)
y_train_subset=np.array(y_train_subset, dtype=np.float32)
reshaped_labels_tensor = torch.tensor(y_train_subset, dtype=torch.float32)

In [9]:
X_train_reshaped = np.array(reshaped_images, dtype=np.float32)
y_train_subset=np.array(y_train_subset, dtype=np.float32)

# Reshape the input images to include the channel dimension
reshaped_images_array = np.expand_dims(X_train_reshaped, axis=-1)
reshaped_images_array = np.tile(reshaped_images_array, (1, 1, 1, 3))

In [10]:
reshaped_images_array.shape

(100, 227, 227, 3)

In [ ]:
model_2.fit(reshaped_images_array, y_train_subset, epochs=5)

Epoch 1/5


C:\Users\EmiliaGachowetz-Gepp\anaconda3\Lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [ ]:
reshaped_images_test = image_reshape(X_test_subset)
X_test_reshaped = np.array(reshaped_images_test, dtype=np.float32)
y_test_subset=np.array(y_test_subset, dtype=np.float32)

# Reshape the input images to include the channel dimension
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=-1)
X_test_reshaped = np.tile(reshaped_images_array, (1, 1, 1, 3))

test_loss, test_acc = model_2.evaluate(X_test_reshaped,  y_test_subset, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
#predictions
probability_model = tf.keras.Sequential([model_2, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(X_test_reshaped)
print(predictions[0])
#A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 27 different logos.
#You can see which label has the highest confidence value by calling:
print(f'most probable logo is',np.argmax(predictions[0]))

In [ ]:
#the model is most confident that this image is of class XX
#Examining the test label shows that this classification is correct?False?:

y_test_subset[0]